# 15 스트림 처리와 점진적 입출력

* 스칼라로 배우는 함수형 프로그래밍 - 15장
* 김무성

# 차례 

* 15.1 명령식 입출력의 문제점을 보여주는 예제 하나
* 15.2 간단한 스트림 변환기
    - 15.2.1 처리 공정의 생성
    - 15.2.2 처리 공정의 합성과 추가
    - 15.2.3 파일 처리
* 15.3 확장 가능한 처리 공정 형식
    - 15.3.1 공급원
    - 15.3.2 자원 안정성 보장
    - 15.3.3 단일 입력 처리 공정
    - 15.3.4 다중 입력 스트림
    - 15.3.5 싱크 
    - 15.3.6 효과 있는 채널
    - 15.3.7 동적 자원 할당
* 15.4 응용
* 15.5 요약

# 15.1 명령식 입출력의 문제점을 보여주는 예제 하나

<img src="figures/cap15.1.jpg" width=700 />

In [ ]:
lines.zipWidthIndex.exists(_._2 + 1 >= 40000)

In [ ]:
lines.filter(!_.trim.isEmpty).zipWidthIndex.exists(_._2 + 1 >= 40000)

In [ ]:
x
lines.filter(!_.trim.isEmpty).
            take(40000).
            map(_.head).
            indexOfSlice("abracadabra".toList)

In [ ]:
Stream[String] x

In [ ]:
IO[Stream[String]]

In [ ]:
def lines(filename: String) : IO[Stream[String]] = IO {
    val src = io.Source.fromFile(filename)
    src.getLines.toStream append { src.close; Stream.empty}
}

# 15.2 간단한 스트림 변환기

* 15.2.1 처리 공정의 생성
* 15.2.2 처리 공정의 합성과 추가
* 15.2.3 파일 처리

<img src="figures/cap15.2.jpg" width=700 />

In [ ]:
def apply(s: Stream[I]) : Stream[0] = this match {
    case Halt() => Stream()
    case Await(recv) => s match {
        case h #:: t => rcev(Some(h))(t)
        case xs => recv(None)(xs) // stream이 비었음
    }
    case Emit(h, t) => h #:: t(s)
}

## 15.2.1 처리 공정의 생성

In [ ]:
def liftOne[I,O](f: I => O): Process[I,O] =
    Await {
        case Some(i) => Empty(f(i))
        case None => Halt()
    }

<img src="figures/cap15.3.jpg" width=700 />

In [ ]:
def lift[I,O](f: I => O) : Process[I,O] = liftOne(f).repeat

In [ ]:
scals> val ones = lift((_:Unit) => 1)(units)
ones: Stream[Int] = Stream(1, ?)

In [ ]:
def filter[I](p: I => Boolean) : Process[I,I] =
    Await[I,I] {
        case Some(i) if p(i) => emit(i)
        case _ => Halt()
    }.repeat

In [ ]:
scala> val evens = even(Stream(1,2,3,4)).toList
evens : List[Int] = List(2,4)

In [ ]:
def sum: Process[Doube, Double] = {
    def go(acc: Double) : Process[Double,Double] =
        Await {
            case Some(d) => Emit(d+acc, go(d+acc))
            case None => Halt()
        }
    go(0.0)
}

In [ ]:
scala> val s = sum(Stream(1.0, 2.0, 3.0, 4.0)).toList
s: List[Double] = List(1.0, 3.0, 6.0, 10.0)

In [ ]:
def loop[S,I,O](z: S)(f: (I,S) => (O,S)): Process[I,O] =
    await((i: I) => f(i,z) match {
        case (o,s2) => emit(o, loop(s2)(f))
    })

## 15.2.2 처리 공정의 합성과 추가

In [ ]:
f |> g

In [ ]:
def |> [O2](p2: Process[O,O2]) : Process[I,O2]

In [ ]:
filter(_ % 2 ==0) |> lift(_ + 1)

In [ ]:
def map[O2](f: O => O2): Process[I,O2] = this |> lift(f)

In [ ]:
def ++(p: => Process[I,O]): Process[I,O] = this match {
    case Halt() => p
    case Emit(h,t) => Emit(h, t ++ p)
    case Await(recv) => Await(recv andThen (_ ++ p))
}

In [ ]:
def monad[I]: Monad[({type f[x] = Process[I,x]})#f] =
    new Monad[({type f[x] = Process[I,x]})#f] {
        def unit[0](o: => O): Process[I,x] = Emit(o)
        def flatMap[o,o2](p:Process[I,O])(
                        f: O => Process[I,O2]) : Process[I,O2] =
                    p flatMat f
    }

## 15.2.3 파일 처리

<img src="figures/cap15.4.jpg" width=700 />

In [ ]:
processFile(f, count |> exists(_ > 40000), flase)(_ || _)

# 15.3 확장 가능한 처리 공정 형식

* 15.3.1 공급원
* 15.3.2 자원 안정성 보장
* 15.3.3 단일 입력 처리 공정
* 15.3.4 다중 입력 스트림
* 15.3.5 싱크 
* 15.3.6 효과 있는 채널
* 15.3.7 동적 자원 할당

<img src="figures/cap15.5.jpg" width=700 />

<img src="figures/cap15.6.jpg" width=700 />

<img src="figures/cap15.7.jpg" width=700 />

## 15.3.1 공급원

In [ ]:
case Class Await[A,O] (
    req: IO[A],
    recv: Either[Trowalbe, A] => Process[IO,O]
) extends Process[IO, O]

<img src="figures/cap15.8.jpg" width=700 />

## 15.3.2 자원 안정성 보장

In [ ]:
runlog { lines("nmaes.txt") |> take(5)}

## 15.3.3 단일 입력 처리 공정

<img src="figures/cap15.9.jpg" width=700 />

## 15.3.4 다중 입력 스트림

<img src="figures/cap15.10.jpg" width=700 />

## 15.3.5 싱크 

## 15.3.6 효과 있는 채널

## 15.3.7 동적 자원 할당

# 15.4 응용

# 15.5 요약

# 참고자료

* [1] 스칼라로 배우는 함수형 프로그래밍 - http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791185890180
* [2] 책 예제 코드 github - https://github.com/fpinscala/fpinscala